In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import torch.nn.functional as F
import os

In [2]:
# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'{device} selected')

cpu selected


In [3]:
emotions_dict = {'ang': 0,
                'hap': 1,
                'sad': 2,
                'fea': 3,
                'sur': 4,
                'neu': 5
}

In [4]:
x_test_text = pd.read_csv('../data/text_test.csv')
y_test_text = x_test_text['label']

x_test_audio = pd.read_csv('../data/audio_test_v0.csv')
y_test_audio = x_test_audio['label']

y_test = y_test_audio  # since y_train_audio == y_train_text

In [5]:
# Concatenate the 'transcription' columns directly
transcription_test = x_test_text['transcription']

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

# Apply TfidfVectorizer
x_test_text = tfidf.fit_transform(transcription_test).toarray()

print(x_test_text.shape)

(1960, 422)


In [6]:
combined_x_test = np.concatenate((np.array(x_test_audio[x_test_audio.columns[2:]]), x_test_text), axis=1)

print(combined_x_test.shape)

(1960, 698)


In [7]:
# need to make dummy input channel for CNN input feature tensor
X_test = np.expand_dims(combined_x_test,1)

# convert emotion labels from list back to numpy arrays for PyTorch to work with
y_test = np.array(y_test)

In [26]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet1D(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet1D, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, dropout=0.2)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2, dropout=0.2)
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=1, dropout=0.2)
        self.layer4 = self._make_layer(block, 128, num_blocks[3], stride=2, dropout=0.2)
        self.linear = nn.Linear(128*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride, dropout):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        layers.append(nn.Dropout(dropout))  # Add dropout layer after the block
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool1d(out, out.size(2))
        out = out.view(out.size(0), -1)
        out_logits = self.linear(out)
        out_softmax = F.softmax(out_logits, dim=1)
        return out_logits, out_softmax

In [27]:
# define loss function: CrossEntropyLoss()
def criterion(predictions, targets):
    return nn.CrossEntropyLoss()(input=predictions, target=targets)

In [30]:
def make_validate_fnc(model,criterion):
    def validate(X,Y):

        # don't want to update any network parameters on validation passes: don't need gradient
        # wrap in torch.no_grad to save memory and compute in validation phase:
        with torch.no_grad():

            # set model to validation phase i.e. turn off dropout and batchnorm layers
            model.eval()

            # get the model's predictions on the validation set
            output_logits, output_softmax = model(X)
            predictions = torch.argmax(output_softmax,dim=1)

            # calculate the mean accuracy over the entire validation set
            accuracy = torch.sum(Y==predictions)/float(len(Y))

            # compute error from logits (nn.crossentropy implements softmax)
            loss = criterion(output_logits,Y)

        return loss.item(), accuracy*100, predictions
    return validate

In [31]:
optimizer = None

def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch

In [38]:
# pick load folder
load_folder = '../data/model_checkpoints'

# pick the epoch to load
epoch = '004'

model_name = f'parallel_CNN1D_FINAL-{epoch}.pkl'

# make full load path
load_path = os.path.join(load_folder, model_name)

## instantiate empty model and populate with params from binary
#model = CNN1D(len(emotions_dict))
model = ResNet1D(BasicBlock, [2, 2, 2, 2], num_classes=len(emotions_dict))
#optimizer = torch.optim.SGD(model.parameters(),lr=0.001, weight_decay=1e-4, momentum=0.4)
load_checkpoint(optimizer, model, load_path)

print(f'Loaded model from {load_path}')

Loaded model from ../data/model_checkpoints\parallel_CNN1D_FINAL-004.pkl


In [39]:
# Move the model to GPU if available
model.to(device)

# reinitialize validation function with model from chosen checkpoint
validate = make_validate_fnc(model,criterion)

# Convert to tensors
X_test_tensor = torch.tensor(X_test,device=device).float()
y_test_tensor = torch.tensor(y_test,dtype=torch.long,device=device)

test_loss, test_acc, predicted_emotions = validate(X_test_tensor,y_test_tensor)

print(f'Test accuracy is {test_acc:.2f}%')

Test accuracy is 32.35%
